# Tellmotorveg og motortrafikkveg 

Teller lengde på motorveg og motortrafikkveg - og "vanlig veg"

Bruker KOSTRA regler og segmentering av vegnett + vegobjekt 595 Motorveg


In [21]:
import pandas as pd
import geopandas as gpd
from shapely import wkt 

import STARTHER
import nvdbapiv3 
import nvdbgeotricks 
import segmentering

In [2]:
vegnettfilter = { 'trafikantgruppe' : 'K', 
                     'vegsystemreferanse' : 'Ev,Rv,Fv,Kv', 
                     'adskiltelop' : 'med,nei',
                     'veglenketype' : 'hoved',
                     'sideanlegg' : False }


In [3]:
vegdf = pd.DataFrame( nvdbapiv3.nvdbVegnett( filter=vegnettfilter ).to_records())

Eksport av 811647 vegsegmenter kommer til å ta tid...
Vegsegment 1000 av 811647
Vegsegment 5000 av 811647
Vegsegment 10000 av 811647
Vegsegment 20000 av 811647
Vegsegment 30000 av 811647
Vegsegment 40000 av 811647
Vegsegment 50000 av 811647
Vegsegment 60000 av 811647
Vegsegment 70000 av 811647
Vegsegment 80000 av 811647
Vegsegment 90000 av 811647
Vegsegment 100000 av 811647
Vegsegment 110000 av 811647
Vegsegment 120000 av 811647
Vegsegment 130000 av 811647
Vegsegment 140000 av 811647
Vegsegment 150000 av 811647
Vegsegment 160000 av 811647
Vegsegment 170000 av 811647
Vegsegment 180000 av 811647
Vegsegment 190000 av 811647
Vegsegment 200000 av 811647
Vegsegment 210000 av 811647
Vegsegment 220000 av 811647
Vegsegment 230000 av 811647
Vegsegment 240000 av 811647
Vegsegment 250000 av 811647
Vegsegment 260000 av 811647
Vegsegment 270000 av 811647
Vegsegment 280000 av 811647
Vegsegment 290000 av 811647
Vegsegment 300000 av 811647
Vegsegment 310000 av 811647
Vegsegment 320000 av 811647
Vegsegm

In [5]:
col = ['veglenkesekvensid', 'startposisjon', 'sluttposisjon',
       'kortform',  'detaljnivå', 'typeVeg', 
        'feltoversikt', 'lengde', 'fylke', 'kommune',
       'medium', 'vref',
       'vegkategori', 'trafikantgruppe', 'adskilte_lop', 'geometry' ]
veg = gpd.GeoDataFrame( vegdf, geometry=vegdf['geometri'].apply( wkt.loads), crs=5973  )
veg = veg[ col ]

### Laster ned Motorveg-objekt og ekstra strengt KOSTRA-filter 

Men litt strengere - vi tar vekk ALLE kryssdeler 


In [8]:
motordf = pd.DataFrame( nvdbapiv3.nvdbFagdata(595, filter={'vegsystemreferanse' : 'Ev,Rv,Fv,Kv'}).to_records())

In [19]:
motordf = motordf[ ~motordf['vref'].str.contains( 'KD' ) ]
motordf = motordf[ ~motordf['vref'].str.contains( 'SA' ) ]
motordf = motordf[ motordf['adskilte_lop'] != 'Mot' ] 
motordf = motordf[ motordf['veglenkeType'] != 'KONNEKTERING' ] 

In [20]:
motordf['veglenkeType'].value_counts()

veglenkeType
HOVED    1208
Name: count, dtype: int64

In [24]:
motordf = gpd.GeoDataFrame( motordf, geometry=motordf['geometri'].apply( wkt.loads ), crs=5973) 

In [25]:
motordf = motordf.drop( columns=['Vedtaksnummer', 'Arkivnummer', 'geometri'] )

In [30]:
motordf.groupby( 'vegkategori').agg( {'segmentlengde' : 'sum' } )

,segmentlengde
vegkategori,
E,951140.802
F,8567.235
R,121589.401


# Segmentering

In [31]:
segm = segmentering.segmenter( veg, motordf ) 

Segmentering av 811647 datarader med veg og 1 typer fagdata med ialt 1208 datarader
Segmentering: Behandler vegsegment 1 av 811647 ( 0% ) tidsbruk 0:00:00
Segmentering: Behandler vegsegment 10 av 811647 ( 0% ) tidsbruk 0:00:00
Segmentering: Behandler vegsegment 100 av 811647 ( 0% ) tidsbruk 0:00:00
Segmentering: Behandler vegsegment 500 av 811647 ( 0% ) tidsbruk 0:00:01
Segmentering: Behandler vegsegment 1000 av 811647 ( 0% ) Estimert ferdig om: 0:33:35 
Segmentering: Behandler vegsegment 10000 av 811647 ( 1% ) Estimert ferdig om: 0:29:43 

Degenert tilfelle (f.eks stedfesting MOT?) veglenkesekvens=22153 kommune=4202
	Vegnett=(0.0,0.03037244), fagdata=(0.00301109, 0.03037244) 
	ev18 s11d1 m0-2986 
	POINT (125643.605 6485928.875) - POINT (127140.653 6487913.239)

Segmentering: Behandler vegsegment 20000 av 811647 ( 2% ) Estimert ferdig om: 0:29:16 
Segmentering: Behandler vegsegment 30000 av 811647 ( 4% ) Estimert ferdig om: 0:28:56 
Segmentering: Behandler vegsegment 40000 av 811647 ( 

In [39]:
segm['Motorvegtype'] = segm['Motorvegtype'].fillna( 'Vanlig veg' ) 

In [47]:
segm['Lengde (km)'] = segm['geometry'].apply( lambda x : x.length / 1000 ) 

In [49]:
resultat = segm.groupby( ['vegkategori', 'Motorvegtype']).agg( {'Lengde (km)' : 'sum' } ).reset_index()

In [55]:
nvdbgeotricks.skrivexcel( 'motorveger.xlsx',  resultat.sort_values( by='Motorvegtype' ) )

skrev 1 faner til motorveger.xlsx 


In [54]:
nvdbgeotricks.skrivexcel?

Signature:
nvdbgeotricks.skrivexcel(
    filnavn,
    dataFrameListe,
    sheet_nameListe=[],
    indexListe=[],
    slettgeometri=True,
)
Docstring:
Skriver liste med dataFrame til excel, med kolonnebredde=lengste element i header eller datainnhold

Skipper kolonner med navn "geometri" eller "geometry" 

ARGUMENTS
    filnavn : Navn på excel-fil 

    dataFrameListe : Liste med dataframe, eller en enkelt dataFrame / geodataframe 

KEYWORDS
    sheet_nameListe : [] Liste med navn på fanene i exel-arket. Hvis tom liste brukes Fane1, Fane2...

    indexListe : [] Angir om index skal med som første kolonne(r), liste med True eller False. Default: Uten index. 

    slettgeometri : True . Sletter geometrikolonner 
File:      /mnt/c/data/leveranser/nvdbapi-V3/nvdbgeotricks.py
Type:      function

In [53]:
segm[ (segm['vegkategori'] == 'F') & (segm['Motorvegtype'] == 'Motorveg') ]

,veglenkesekvensid,startposisjon,sluttposisjon,vref,feltoversikt,fylke,kommune,vegkategori,geometry,versjon,Motorvegtype,segmentlengde,segmentretning,stedfesting_retning,Lengde (km)
777125,2518835,0.949416,0.951759,fv557 s1d1 m9309-9320,1,46,4601,F,"LINESTRING Z (-36295.787 6732312.429 34.025, -...",1.0,Motorveg,227.686,MED,NaN,0.011432
777126,2518835,0.951759,0.996068,fv557 s1d1 m9320-9536,1,46,4601,F,"LINESTRING Z (-36294.09 6732323.55 34.055, -36...",1.0,Motorveg,227.686,MED,NaN,0.216429
